In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/COMP4641\ Project

/content/drive/My Drive/COMP4641 Project


In [0]:
import pandas as pd
import numpy as np
import math
import ast

In [0]:
%ls

 business.json   LSTM/                    preparedata/             user.json
 checkin.json    newnotebooks/           'race data'/
 data/           notebooks/               review.json
 final/         'NOT to submit.gslides'  'sentimental analysis'/


In [0]:
final_sentiment = pd.read_csv('final/rate_sentiment_final.csv')
final_sentiment = final_sentiment.drop(['Unnamed: 0'],axis=1)

In [12]:
final_sentiment

,business_id,name,stars,review_count,is_open,categories,processed_city,user_stars,user_fans,checkin_dates,reviews2013,reviews2014,reviews2015,rate_mid_2013,rate_start_2014,rate_mid_2014,rate_start_2015,rate_mid_2015,reviews2013_sentiment_bin_score,reviews2014_sentiment_bin_score,reviews2015_sentiment_bin_score
0,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,3.0,18,1,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"[3.76, 2.55, 3.68, 3.17, 3.68, 3.68, 2.31, 3.6...","[0, 5, 29, 0, 2, 0, 0, 92, 26, 0, 3, 0, 17, 17...","['2013-01-01 08:38:32', '2013-01-07 01:26:01',...",['Taco Bell has really improved over the years...,"[""Roses are Red\nViolets are Blue,\nI'd be tru...",[],-5,-3,0,-1,-4,"[1, 0]",[1],[]
1,q2KtxnPa6rPSpAlMbg9l8g,Nothing Bundt Cakes,4.0,174,1,"Desserts, Food, Cupcakes, Bakeries",Scottsdale,"[3.82, 3.55, 4.13, 5.0, 3.63, 5.0, 4.11, 4.75,...","[4, 1, 18, 0, 0, 0, 35, 0, 1, 43, 0, 10, 2, 7,...","['2013-01-11 21:29:14', '2013-01-22 22:08:56',...",['Thank you Nothing Bundt Cakes for donating m...,"[""I called ten other bakers and this is the on...","['I placed an on-line order for an 8"" chocolat...",-5,7,-6,-2,6,"[1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,44YFU284Z3KDEy25QyVoUw,Nee House Chinese Restaurant,3.5,269,1,"Chinese, Restaurants",Phoenix,"[4.7, 4.25, 3.93, 4.07, 3.57, 3.33, 4.13, 3.5,...","[0, 0, 111, 10, 4, 5, 0, 0, 11, 5, 3, 1, 1, 1,...","['2013-01-13 01:30:16', '2013-01-17 01:16:37',...","[""We eat here on a regular basis. It's like t...",['My mom loves this restaurant ! Good food ! B...,"[""3.5 *s. Had west lake beef soup (pretty goo...",10,4,-6,-2,0,"[1, 1, -1, 1, 0, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1...","[1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","[1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 0, 1, 1, 1..."
3,ysfeDcLAYk_MVzfBmyiSTg,Vals Getaway DES Cafeteria,3.0,14,1,"Restaurants, Food, Cafes",Phoenix,"[3.25, 2.75, 2.63, 4.18, 4.15, 4.41, 3.82, 4.5...","[0, 2, 9, 0, 0, 2, 0, 0, 9, 9, 0, 5, 2, 0]","['2013-06-21 19:33:23', '2014-03-12 22:42:46',...",[],['The employees are nice and accommodating and...,[],-1,3,1,-3,-1,[],"[1, 1, 1, 1, 1]",[]
4,nsNONDHbV7Vudqh21uicqw,Salt Cellar,3.5,526,1,"Seafood, Restaurants",Scottsdale,"[1.5, 3.7, 1.0, 2.17, 3.76, 3.33, 4.0, 3.37, 4...","[0, 4, 0, 0, 1, 0, 1, 0, 3, 30, 0, 0, 2, 0, 0,...","['2013-01-06 00:15:01', '2013-01-09 01:25:09',...","[""As a lover of seafood I have been wanting to...",['Came back to this famous place after years o...,"['Poor service, had to sit there and ask my wi...",-5,-12,14,10,-16,"[1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1...","[1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7977,3xI0ToitrAJkrUSH4gbmCA,Subway,2.0,14,1,"Restaurants, Sandwiches, Fast Food",Phoenix,"[4.62, 4.57, 4.62, 2.76, 3.02, 1.0, 2.24, 3.96...","[0, 1, 5, 1, 7, 0, 0, 2, 0, 0, 0, 0, 0, 0]","['2013-05-23 12:26:54', '2013-05-26 22:07:59',...",['THE WORST SUBWAY EXPERIENCE EVER!\n\nFirst o...,['I love this location! Its close to lots of s...,"[""Don't go to this location. There's one om 48...",0,-1,-3,3,-2,"[-1, -1]","[1, 1, -1]",[1]
7978,cILk7PnJBxNsMmGhQU2cyA,Lola's,3.5,28,0,"Coffee & Tea, Restaurants, Food, Tapas Bars",Phoenix,"[3.88, 4.04, 2.0, 4.04, 4.22, 2.75, 3.68, 3.63...","[0, 2, 0, 38, 0, 2, 1, 7, 0, 1, 4, 0, 1, 44, 2...","['2014-03-30 15:03:14', '2014-04-01 19:39:41',...",[],"[""I learned about something new today. I was a...",[],0,82,-45,-37,0,[],"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[]
7979,mk_OXeS7_zcfYlaEEFYWvA,Safeway,3.5,23,1,"Grocery, Food",Phoenix,"[2.69, 3.97, 2.87, 3.72, 3.48, 4.25, 3.92, 4.8...","[0, 33, 4, 1, 21, 0, 22, 8, 2, 4, 0, 23, 23, 8...","['2013-01-13 19:49:09', '2013-01-23 18:19:18',...",['I just came here for sodas and water. My las...,['I love this store and the staff here. Well l...,"[""Stopped 

In [0]:
business_json = pd.read_json('business.json',lines=True)

In [0]:
business_ariz = business_json[business_json['business_id'].isin(final_sentiment['business_id'].to_list())]

In [15]:
business_ariz.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [0]:
business_with_lat_long = pd.merge(final_sentiment, business_ariz.drop(['name', 'address', 'city', 'state', 'postal_code','stars', 'review_count', 'is_open','attributes', 'categories', 'hours'],axis=1), left_on='business_id', right_on='business_id')

In [0]:
business_with_lat_long.to_csv('final/rate_sentiment_lat_long.csv',header=True)

In [9]:
%ls final/

rate_of_checkin_per_6_months.csv  rate_sentiment_lat_long.csv
rate_sentiment_final.csv


### Computing the Neighbours
Neighbour distance = 1km

In [0]:
all_df = pd.read_csv('final/rate_sentiment_lat_long.csv').drop(['Unnamed: 0'],axis=1)

In [5]:
all_df['processed_city'].unique()

array(['Phoenix', 'Scottsdale', 'Mesa', 'Tempe', 'Maricopa County'],
      dtype=object)

In [0]:
phoenix_all = all_df[all_df['processed_city']=='Phoenix'].reset_index().drop(['index'],axis=1)
scott_all = all_df[all_df['processed_city']=='Scottsdale'].reset_index().drop(['index'],axis=1)
mesa_all = all_df[all_df['processed_city']=='Mesa'].reset_index().drop(['index'],axis=1)
tempe_all = all_df[all_df['processed_city']=='Tempe'].reset_index().drop(['index'],axis=1)
maricopa_all = all_df[all_df['processed_city']=='Maricopa County'].reset_index().drop(['index'],axis=1)

In [0]:
def getrad(x):
  return x * math.pi/180;

def getDistance(p1_lat,p1_lng,p2_lat,p2_lng):
  R = 6378137
  dLat = getrad(p2_lat-p1_lat)
  dLong = getrad(p2_lng-p1_lng)
  a = math.sin(dLat/2)*math.sin(dLat/2) + math.cos(getrad(p1_lat)) * math.cos(getrad(p2_lat))* math.sin(dLong/2) * math.sin(dLong/2)
  c = 2*math.atan2(math.sqrt(a),math.sqrt(1-a))
  d = R*c
  return d


In [8]:
phoenix_all[:2]

,business_id,name,stars,review_count,is_open,categories,processed_city,user_stars,user_fans,checkin_dates,reviews2013,reviews2014,reviews2015,rate_mid_2013,rate_start_2014,rate_mid_2014,rate_start_2015,rate_mid_2015,reviews2013_sentiment_bin_score,reviews2014_sentiment_bin_score,reviews2015_sentiment_bin_score,latitude,longitude
0,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,3.0,18,1,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"[3.76, 2.55, 3.68, 3.17, 3.68, 3.68, 2.31, 3.6...","[0, 5, 29, 0, 2, 0, 0, 92, 26, 0, 3, 0, 17, 17...","['2013-01-01 08:38:32', '2013-01-07 01:26:01',...",['Taco Bell has really improved over the years...,"[""Roses are Red\nViolets are Blue,\nI'd be tru...",[],-5,-3,0,-1,-4,"[1, 0]",[1],[],33.495194,-112.028588
1,44YFU284Z3KDEy25QyVoUw,Nee House Chinese Restaurant,3.5,269,1,"Chinese, Restaurants",Phoenix,"[4.7, 4.25, 3.93, 4.07, 3.57, 3.33, 4.13, 3.5,...","[0, 0, 111, 10, 4, 5, 0, 0, 11, 5, 3, 1, 1, 1,...","['2013-01-13 01:30:16', '2013-01-17 01:16:37',...","[""We eat here on a regular basis. It's like t...",['My mom loves this restaurant ! Good food ! B...,"[""3.5 *s. Had west lake beef soup (pretty goo...",10,4,-6,-2,0,"[1, 1, -1, 1, 0, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1...","[1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","[1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 0, 1, 1, 1...",33.613020,-111.977036


In [9]:
getDistance(33.495194,-112.028588,33.613020,-111.977036)

13961.018976082032

In [0]:
def getneighbours_index(df,neighbour_definer=1000):
  distances = []
  for i in range(len(df)):
    dist = []
    for j in range(len(df)):
      if i == j:
        continue
      index = (i+j)%len(df)
      distance = getDistance(df['latitude'][i],df['longitude'][i],df['latitude'][index],df['longitude'][index])
      if distance < neighbour_definer:
        dist.append(index)

    distances.append(dist)

  num_neighbours = [len(distances[i]) for i in range(len(distances))]

  print("Done with loop")
  df['num_neighbours'] = num_neighbours
  df['neighbour_index'] = distances
  return df

In [11]:
phoenix_neigh = getneighbours_index(phoenix_all)
scott_neigh = getneighbours_index(scott_all)
mesa_neigh = getneighbours_index(mesa_all)
tempe_neigh = getneighbours_index(tempe_all)
maricopa_neigh = getneighbours_index(maricopa_all)

KeyboardInterrupt: ignored

In [0]:
(phoenix_neigh['categories'][0]).split(', ')

In [0]:
def get_neigh_entropy(df):
  common_neighbours = []
  for i in range(len(df)):
    list_i = (df['categories'][i].split(', '))
    list_neigh = df['neighbour_index'][i]
    cat_count_list = []
    for cat in list_i:
      count = 0
      for j in range(len(list_neigh)):
        list_j = df['categories'][list_neigh[j]].split(', ')
        if cat in list_j:
          count+=1
      cat_count_list.append(count)

    common_neighbours.append(cat_count_list)


    #common_neighbours.append(common_cat_neigh)

  df['neighbours_with_category'] = common_neighbours
  return df


In [0]:
phoenix_neigh_cat = get_neigh_entropy(phoenix_neigh.copy())
scott_neigh_cat = get_neigh_entropy(scott_neigh.copy())
mesa_neigh_cat = get_neigh_entropy(mesa_neigh.copy())
tempe_neigh_cat = get_neigh_entropy(tempe_neigh.copy())
maricopa_neigh_cat = get_neigh_entropy(maricopa_neigh.copy())

## Entropy computing, defined as

$f_{entropy}= \sum_{\gamma \in \Gamma}\frac{N_{\gamma}(r)}{N(r)} \times log{\frac{N_{\gamma}(r)}{N(r)}}$

where 
- $N_{\gamma}(r)$ is defined as the number of neighbours with same restaurant category
- $N(r)$ is defined as the total number of neighbours

In [0]:
def calc_entropy(total_neigh,same_cat_neigh):
  entropy = 0
  if total_neigh is 0:
    return 0
  for neigh in same_cat_neigh:
    if neigh is 0:
      continue
    num = neigh/float(total_neigh)
    log_val = math.log10(neigh/total_neigh)
    entropy += (num*log_val)

  return entropy * (-1)

In [0]:
phoenix_neigh_cat['entropy'] = phoenix_neigh_cat.apply(lambda row: calc_entropy(row['num_neighbours'],row['neighbours_with_category']), axis=1)
scott_neigh_cat['entropy'] = scott_neigh_cat.apply(lambda row: calc_entropy(row['num_neighbours'],row['neighbours_with_category']), axis=1)
mesa_neigh_cat['entropy'] = mesa_neigh_cat.apply(lambda row: calc_entropy(row['num_neighbours'],row['neighbours_with_category']), axis=1)
tempe_neigh_cat['entropy'] = tempe_neigh_cat.apply(lambda row: calc_entropy(row['num_neighbours'],row['neighbours_with_category']), axis=1)
maricopa_neigh_cat['entropy'] = maricopa_neigh_cat.apply(lambda row: calc_entropy(row['num_neighbours'],row['neighbours_with_category']), axis=1)

## Competitveness computing, defined as

$f_{com}= \frac{N_{\gamma(r)}(r)}{N(r)}$

where 
- $N_{\gamma(r)}(r)$ is defined as the number of neighbours with same restaurant category
- $N(r)$ is defined as the total number of neighbours

In [0]:
def get_neigh_competitveness(df):
  no_of_same_cat_neighbours = []
  neighbours_same_cat_index = []
  total_neigh_checkin_list = []
  for i in range(len(df)):
    list_i_cat = df['categories'][i].split(', ')
    count=0
    neigh_cat_list = []
    total_neigh_cat_checkin = len(ast.literal_eval(df['checkin_dates'][i]))
    for neighbour in df['neighbour_index'][i]:
      neighbour_cat = df['categories'][neighbour].split(', ')
      have_common_cat = any(check in list_i_cat for check in neighbour_cat) 

      if have_common_cat:
        count+=1
        total_neigh_cat_checkin += len(ast.literal_eval(df['checkin_dates'][neighbour]))
        neigh_cat_list.append(neighbour)
    
    total_neigh_checkin_list.append(total_neigh_cat_checkin)
    neighbours_same_cat_index.append(neigh_cat_list)
    no_of_same_cat_neighbours.append(count)

  df['neighbours_same_category'] = no_of_same_cat_neighbours
  df['neighbours_same_cat_index'] = neighbours_same_cat_index
  df['total_neighbour_checkins'] = total_neigh_checkin_list
  return df


In [0]:
phoenix_neigh_cat2 = get_neigh_competitveness(phoenix_neigh_cat.copy())
scott_neigh_cat2 = get_neigh_competitveness(scott_neigh_cat.copy())
mesa_neigh_cat2 = get_neigh_competitveness(mesa_neigh_cat.copy())
tempe_neigh_cat2 = get_neigh_competitveness(tempe_neigh_cat.copy())
maricopa_neigh_cat2 = get_neigh_competitveness(maricopa_neigh_cat.copy())

In [0]:
phoenix_neigh_cat2['competitiveness'] = phoenix_neigh_cat2.apply(lambda row: row['neighbours_same_category']/float(row['num_neighbours']),axis=1)
scott_neigh_cat2['competitiveness'] = scott_neigh_cat2.apply(lambda row: row['neighbours_same_category']/float(row['num_neighbours']),axis=1)
mesa_neigh_cat2['competitiveness'] = mesa_neigh_cat2.apply(lambda row: row['neighbours_same_category']/float(row['num_neighbours']),axis=1)
tempe_neigh_cat2['competitiveness'] = tempe_neigh_cat2.apply(lambda row: row['neighbours_same_category']/float(row['num_neighbours']),axis=1)
maricopa_neigh_cat2['competitiveness'] = maricopa_neigh_cat2.apply(lambda row: row['neighbours_same_category']/float(row['num_neighbours']),axis=1)

In [0]:
phoenix_neigh_cat2['peer_popularity'] = phoenix_neigh_cat2.apply(lambda row: len(ast.literal_eval(row['checkin_dates']))/float(row['total_neighbour_checkins']/float(row['num_neighbours'])),axis=1)
scott_neigh_cat2['peer_popularity'] = scott_neigh_cat2.apply(lambda row: len(ast.literal_eval(row['checkin_dates']))/float(row['total_neighbour_checkins']/float(row['num_neighbours'])),axis=1)
mesa_neigh_cat2['peer_popularity'] = mesa_neigh_cat2.apply(lambda row: len(ast.literal_eval(row['checkin_dates']))/float(row['total_neighbour_checkins']/float(row['num_neighbours'])),axis=1)
tempe_neigh_cat2['peer_popularity'] = tempe_neigh_cat2.apply(lambda row: len(ast.literal_eval(row['checkin_dates']))/float(row['total_neighbour_checkins']/float(row['num_neighbours'])),axis=1)
maricopa_neigh_cat2['peer_popularity'] = maricopa_neigh_cat2.apply(lambda row: len(ast.literal_eval(row['checkin_dates']))/float(row['total_neighbour_checkins']/float(row['num_neighbours'])),axis=1)

In [13]:
%ls 

 business.json   LSTM/                    preparedata/             user.json
 checkin.json    newnotebooks/           'race data'/
 data/           notebooks/               review.json
 final/         'NOT to submit.gslides'  'sentimental analysis'/


In [0]:
phoenix_neigh_cat2.to_csv('preparedata/future_work/phoenix_future.csv',header=True)
scott_neigh_cat2.to_csv('preparedata/future_work/scott_future.csv',header=True)
mesa_neigh_cat2.to_csv('preparedata/future_work/mesa_future.csv',header=True)
tempe_neigh_cat2.to_csv('preparedata/future_work/tempe_future.csv',header=True)
maricopa_neigh_cat2.to_csv('preparedata/future_work/maricopa_future.csv',header=True)

## Converting to timeseries

In [0]:
phoenix_df = pd.read_csv('preparedata/future_work/phoenix_future.csv').drop(['Unnamed: 0'],axis=1)
scott_df = pd.read_csv('preparedata/future_work/scott_future.csv').drop(['Unnamed: 0'],axis=1)
mesa_df = pd.read_csv('preparedata/future_work/mesa_future.csv').drop(['Unnamed: 0'],axis=1)
tempe_df = pd.read_csv('preparedata/future_work/tempe_future.csv').drop(['Unnamed: 0'],axis=1)
maricopa_df = pd.read_csv('preparedata/future_work/maricopa_future.csv').drop(['Unnamed: 0'],axis=1)

In [0]:
phoenix_ts_df = pd.read_csv('LSTM/phoenix_lstm_timeseries_data.csv').drop(['Unnamed: 0'],axis=1)
scott_ts_df = pd.read_csv('LSTM/scot_lstm_timeseries_data.csv').drop(['Unnamed: 0'],axis=1)
mesa_ts_df = pd.read_csv('LSTM/mesa_lstm_timeseries_data.csv').drop(['Unnamed: 0'],axis=1)
tempe_ts_df = pd.read_csv('LSTM/tempe_lstm_timeseries_data.csv').drop(['Unnamed: 0'],axis=1)
maricopa_ts_df = pd.read_csv('LSTM/maricopa_lstm_timeseries_data.csv').drop(['Unnamed: 0'],axis=1)

In [0]:
def updated_ts(df_ts,df_future):
  # entropy
  # competitiveness
  # peer_popularity
  # num_neighbours
  entrop = []
  compet = []
  pp = []
  num_neigh = []

  for i in range(len(df_future)):
    entr = df_future['entropy'][i]
    comp = df_future['competitiveness'][i]
    peerpop = df_future['peer_popularity'][i]
    nn = df_future['num_neighbours'][i]

    entrop.extend([entr,entr,entr,entr,entr])
    compet.extend([comp,comp,comp,comp,comp])
    pp.extend([peerpop,peerpop,peerpop,peerpop,peerpop])
    num_neigh.extend([nn,nn,nn,nn,nn])

  df_ts_copy = df_ts.copy()
  df_ts_copy['Entropy'] = entrop
  df_ts_copy['competitiveness'] = compet
  df_ts_copy['peer_popularity'] = pp
  df_ts_copy['density'] = num_neigh
  return df_ts_copy


In [0]:
phoenix_new_ts = updated_ts(phoenix_ts_df,phoenix_df)
scott_new_ts = updated_ts(scott_ts_df,scott_df)
mesa_new_ts = updated_ts(mesa_ts_df,mesa_df)
tempe_new_ts = updated_ts(tempe_ts_df,tempe_df)
maricopa_new_ts = updated_ts(maricopa_ts_df,maricopa_df)

In [0]:
%ls LSTM/future/

In [0]:
phoenix_new_ts.to_csv('LSTM/future/phoenix_ts.csv',header=True)
scott_new_ts.to_csv('LSTM/future/scott_ts.csv',header=True)
mesa_new_ts.to_csv('LSTM/future/mesa_ts.csv',header=True)
tempe_new_ts.to_csv('LSTM/future/tempe_ts.csv',header=True)
maricopa_new_ts.to_csv('LSTM/future/maricopa_ts.csv',header=True)